In [0]:
%pip install -U openai gradio
dbutils.library.restartPython()

In [0]:
from openai import OpenAI
import os
import sys
import utils
import gradio as gr

sys.path.append('/Workspace/Users/vishesh.arya@databricks.com/india-genai-swat/cx-support-chatbot')
DATABRICKS_TOKEN = dbutils.secrets.get(scope = "db-field-eng", key = "va-pat-token")

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints"
)

In [0]:
def get_completion_from_messages(messages, model="va_ext_azure_openai_gpt35", temperature = 0, max_tokens = 2000):
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature = temperature,
    max_tokens=max_tokens,
  )
  
  return completion.choices[0].message.content

#### System of chained prompts for processing the user query

In [0]:
import utils

def process_user_message(user_input, all_messages=[], debug=True):
  delimiter = "```"

  category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
  
  # Step 1: Extract the list of products
  category_and_product_list = utils.read_string_to_list(category_and_product_response)
  if debug: print("Step 1: Extracted list of products.")
    
  # Step 2: If products are found, look them up
  product_information = utils.generate_output_string(category_and_product_list)
  if debug: print("Step 2: Looked up product information.")
  
  # Step 3: Answer the user question
  
  system_message = f"""You are a customer service assistant for a large electronic store. You take in user's questions, delimited by triple backticks (```) and respond in a friendly and helpful tone, with concise answers. Make sure to ask the user relevant follow-up questions. Do not respond to any user questions not related to the context provided under `Relevant product information`"""
  
  messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}. Relevant product information:\n{product_information}"},
    ]
  
  final_response = get_completion_from_messages(all_messages + messages)
  if debug: print("Step 3: Generated response to user question.")

  # Step 4: Ask the model if the response answers the initial user query well
  user_message = f"""
  Customer message: {delimiter}{user_input}{delimiter}
  Agent response: {delimiter}{final_response}{delimiter}
  
  You are a customer service assistant for a large electronic store. Does the response sufficiently answer the question and is based on the context provided under `Relevant product information`?\n
  Relevant product information:\n{product_information}
  
  Respond only with Y or N
  """
  
  messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_message}
    ]
  
  evaluation_response = get_completion_from_messages(messages)
  if debug: print("Step 4: Model evaluated the response.")

  # Step 5: If yes, use this answer; if not, say that you will connect the user to a human
  if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
    if debug: print("Step 5: Model approved the response.")
    return final_response
  else:
    if debug: print("Step 5: Model disapproved the response.")
    neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
    return neg_str

In [0]:
delimiter = "```"

def collect_messages(prompt, history):
  global context
  
  if not history:
    context = []
  
  context.append({'role': 'user', 'content': f"{delimiter}{prompt}{delimiter}"})

  try:
    response = process_user_message(user_input=prompt, all_messages=context, debug=False)
  except Exception as e:
    error_msg = eval(str(e).split(" - ", 1)[1]) if " - " in str(e) else str(e)
    
    if 'external_model_message' in error_msg:
      response = error_msg['external_model_message']['error']['message']
    elif 'input_guardrail' in error_msg:
      response = error_msg['input_guardrail'][0]
    elif 'output_guardrail' in error_msg:
      response = error_msg['output_guardrail'][0]
    else:
      response = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
  
  context.append({'role': 'assistant', 'content': response})
  return response

In [0]:
demo = gr.ChatInterface(
    collect_messages,
    type="messages",
    textbox=gr.Textbox(placeholder="Hi, How can I help you today?", container=False, scale=7),
    title="Customer Support Chatbot",
    description="This chatbot is a demo example of a customer support chatbot for an electronic store",
    examples=[
        "What TV do you have?",
        "Which is the cheapest?",
        "Which is the most expensive?",
        "Tell me about all of its features",
        "Give me comparison of features for the cheapest vs most expensive option in a table format"
    ],
    cache_examples=False,
)

demo.launch(share=True, debug=False)